## Tic Tac Toe

### <u> Description <u> 
In this implementation of a 4x4 tic-tac-toe game, we will use two reinforcement learning (RL) algorithms: Monte Carlo (MC) and Q-Learning. We begin by describing the major components.

- **Agent** <br>
Player 1 and player 2.

- **Environment** <br> 
The board will be initialized as a 4x4 grid containing only zeroes. When player places their piece, the position will be updated with 1 if the move came from player 1 and -1 if the move came from player 2. 

- **State** <br>
The board state (current piece placements and available spaces) of the agent and its opponent. 

- **Actions** <br>
The positions that a player can choose based on the current board state. At each position, players can either play a piece or cannot (the piece is in use by the opponent). Players will take turns placing pieces and will continue until terminal state is reached. The position they place a piece will be randomly selected from the open positions.

- **Terminal state** <br>
Players cannot move anymore (the board is filled and/or a win/lose/draw condition has been reached). 

- **Reward** <br>
The player receives +1 reward if they win, -1 reward if they lose and 0 reward if they draw. 

### <u>Environment<u>

In [5]:
import numpy as np 
import matplotlib.pyplot as plt 
import random 

In [16]:
BOARD_ROWS = 4
BOARD_COLS = 4

PLAYER_X = 1 
PLAYER_0 = 0 

GAME_STATE_X = -1
GAME_STATE_O = 1
GAME_STATE_DRAW = 0
GAME_STATE_NOT_ENDED = 2

states = [] 

##initialize board 
class Environment:
    def __init__ (self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.terminal = False #bool - game signals terminal state
        self.playerSymbol = PLAYER_X
         
    #returns position at specific location 
    def get_position (self, x, y):
        return self.board[x][y] #note: [x][y] == [x,y]
    
    #set position (i.e, update state) at specific index to player's symbol
    def set_position (self, x, y, player_symbol):
        self.board[x][y] = player_symbol
        
#         if ((x > 4) or (y > 4) or (x < 0) or (y < 0)): 
#             print("you're outta bounds sir")
        
    def print_board(self):
        print(self.board)
        
    #save state to array; 
#     def store_state(self):
        
        
    #return if position is open 
    def check_open(self, x, y):
        #position was empty, return T;
        if (self.board[x][y] == 0): 
            return True 
        else: 
            #position was filled, return F 
            return False 
    
    #determine winner; if agent wins, return 1. if opponent wins, return -1. 
    def winner(self):
        
        winner = None 

        #horizontal win -- player gets 4 in a row across
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 4:
                self.terminal = True
                winner = 1
            if sum(self.board[i, :]) == 0:
                self.terminal = True
                winner = -1

        # vertical win -- player gets 4 in a column 
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 4:
                self.terminal = True
                winner = 1
            if sum(self.board[:, i]) == 0:
                self.terminal = True
                winner = -1
        
        # diagonal win 
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS-i-1] for i in range(BOARD_COLS)])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 4:
            self.terminal = True
            winner = 1
        if diag_sum == 0:
            self.terminal = True
            winner = -1
        
        # tie -- no more available positions
        if len(self.open_positions()) == 0:
            self.terminal = True
            winner = 0
      
        # if the game has not ended, simply return nothing 
        self.terminal = False
        print("Reward is: ", winner)
        return winner
            

#     #determine reward based on winner
#     def reward(self): 
#         result = self.winner() 
        
#         #agent won
#         if (result == 1): 
            
#         #opponent won
#         if (result == -1):
            
#         #tie -- no reward 
            
    
    #return an array of open positions in the board
    def open_positions(self): 
        positions = [] 
        for x in range(BOARD_ROWS):
            for y in range(BOARD_COLS):
                if self.board[x,y] == 0:
                    positions.append((x,y))
        return positions
        
    #clear board, reset all positions to 0
    def reset(self): 
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.terminal = False
        self.playerSymbol = AGENT_SYMBOL
    
    #print board
    def show_board(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')

In [18]:
# class Player:
#     def __init__ (self):
#         self.states = [] 
#         self.env = Environment() 
        
#     def chooseAction(self, positions, current_board, symbol): 
        
    

In [19]:
from abc import ABC, abstractmethod
import os
import pickle
import collections
import numpy as np
import random


class Learner(ABC):
    """
    Parent class for Q-learning and SARSA agents.
    Parameters
    ----------
    alpha : float 
        learning rate
    gamma : float
        temporal discounting rate
    eps : float 
        probability of random action vs. greedy action
    eps_decay : float
        epsilon decay rate. Larger value = more decay
    """
    def __init__(self, alpha, gamma, eps, eps_decay=0.):
        # Agent parameters
        self.alpha = alpha
        self.gamma = gamma
        self.eps = eps
        self.eps_decay = eps_decay
        # Possible actions correspond to the set of all x,y coordinate pairs
        self.actions = []
        for i in range(3):
            for j in range(3):
                self.actions.append((i,j))
        # Initialize Q values to 0 for all state-action pairs.
        # Access value for action a, state s via Q[a][s]
        self.Q = {}
        for action in self.actions:
            self.Q[action] = collections.defaultdict(int)
        # Keep a list of reward received at each episode
        self.rewards = []

    def get_action(self, s):
        """
        Select an action given the current game state.
        Parameters
        ----------
        s : string
            state
        """
        # Only consider the allowed actions (empty board spaces)
        possible_actions = [a for a in self.actions if s[a[0]*3 + a[1]] == '-']
        if random.random() < self.eps:
            # Random choose.
            action = possible_actions[random.randint(0,len(possible_actions)-1)]
        else:
            # Greedy choose.
            values = np.array([self.Q[a][s] for a in possible_actions])
            # Find location of max
            ix_max = np.where(values == np.max(values))[0]
            if len(ix_max) > 1:
                # If multiple actions were max, then sample from them
                ix_select = np.random.choice(ix_max, 1)[0]
            else:
                # If unique max action, select that one
                ix_select = ix_max[0]
            action = possible_actions[ix_select]

        # update epsilon; geometric decay
        self.eps *= (1.-self.eps_decay)

        return action

    def save_agent(self, path):
        """ Pickle the agent object instance to save the agent's state. """
        if os.path.isfile(path):
            os.remove(path)
        f = open(path, 'wb')
        pickle.dump(self, f)
        f.close()

    @abstractmethod
    def update(self, s, s_, a, a_, r):
        pass


class Qlearner(Learner):
    """
    A class to implement the Q-learning agent.
    """
    def __init__(self, alpha, gamma, eps, eps_decay=0.):
        super().__init__(alpha, gamma, eps, eps_decay)

    def update(self, s, s_, a, a_, r):
        """
        Perform the Q-Learning update of Q values.
        Parameters
        ----------
        s : string
            previous state
        s_ : string
            new state
        a : (i,j) tuple
            previous action
        a_ : (i,j) tuple
            new action. NOT used by Q-learner!
        r : int
            reward received after executing action "a" in state "s"
        """
        # Update Q(s,a)
        if s_ is not None:
            # hold list of Q values for all a_,s_ pairs. We will access the max later
            possible_actions = [action for action in self.actions if s_[action[0]*3 + action[1]] == '-']
            Q_options = [self.Q[action][s_] for action in possible_actions]
            # update
            self.Q[a][s] += self.alpha*(r + self.gamma*max(Q_options) - self.Q[a][s])
        else:
            # terminal state update
            self.Q[a][s] += self.alpha*(r - self.Q[a][s])

        # add r to rewards list
        self.rewards.append(r)


class SARSAlearner(Learner):
    """
    A class to implement the SARSA agent.
    """
    def __init__(self, alpha, gamma, eps, eps_decay=0.):
        super().__init__(alpha, gamma, eps, eps_decay)

    def update(self, s, s_, a, a_, r):
        """
        Perform the SARSA update of Q values.
        Parameters
        ----------
        s : string
            previous state
        s_ : string
            new state
        a : (i,j) tuple
            previous action
        a_ : (i,j) tuple
            new action
        r : int
            reward received after executing action "a" in state "s"
        """
        # Update Q(s,a)
        if s_ is not None:
            self.Q[a][s] += self.alpha*(r + self.gamma*self.Q[a_][s_] - self.Q[a][s])
        else:
            # terminal state update
            self.Q[a][s] += self.alpha*(r - self.Q[a][s])

        # add r to rewards list
        self.rewards.append(r)

In [9]:
test = Environment()
test.board

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [10]:
print(test.check_open(2,2))

True


In [17]:
test.set_position(2,4,2)
test.board

IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
class Player: 
    def __init__ (self, player): 
#         self.player = p1 
        self.player_symbol = player #player's symbol is either -1 or 1
        
    def place (): 
        probability = random.randint(0,1) 
        if (probability > 0.5): 
            return True
        else:
            return False 